In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service 
#importing libraries
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd


In [2]:
state_links=["https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/astc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile"
]

# Kerala bus details

In [3]:


# URL of the website
URL = "https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    driver = initialize_driver()  # Initialize the driver once

    try:
        for page in range(1,3):  # There are 3 pages
            try:
                load_page(driver, URL)
                
                if page > 1:
                    pagination_tab = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                    )
                    driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                    driver.execute_script("arguments[0].click();", pagination_tab)
                    time.sleep(5)  # Wait for the page to load
                
                all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
                # Iterate over each bus route link and scrape the details
                for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                    bus_details = scrape_bus_details(driver, link, name)
                    if bus_details:
                        all_bus_details.extend(bus_details)
            except Exception as e:
                print(f"Error occurred while accessing page {page}: {str(e)}")

    finally:
        driver.quit()  # Close the driver at the end

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('C:/Users/adith/Downloads/red_bus_data_1.csv', index=False)

In [4]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 235,Swift Deluxe Non AC Air Bus (2+2),15:00,09h 01m,00:01,3.9,INR 801,5 Seats available
1,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 370,AC MULTI AXLE,15:25,06h 55m,22:20,2.7,INR 1099,6 Seats available
2,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 97,Swift Deluxe Non AC Air Bus (2+2),16:00,08h 45m,00:45,2.4,INR 740,4 Seats available
3,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 171,Swift Deluxe Non AC Air Bus (2+2),16:15,09h 30m,01:45,4.1,INR 832,2 Seats available
4,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 553,Super Express Non AC Seater Air Bus (2+2),16:45,08h 40m,01:25,4.1,INR 832,3 Seats available
...,...,...,...,...,...,...,...,...,...,...
656,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...,A1 Travels,A/C Seater / Sleeper (2+1),23:59,07h 01m,07:00,2.2,INR 700,20 Seats available
657,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...,Ubc travels,A/C Semi Sleeper (2+2),17:30,07h 30m,01:00,3.7,INR 700,8 Seats available
658,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...,Jacobs Travels,A/C Semi Sleeper (2+2),18:30,08h 00m,02:30,3.1,INR 800,34 Seats available
659,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...,Aadithya Travels,A/C Seater/Sleeper (2+1),19:40,06h 05m,01:45,2.6,INR 1500,1 Seat available


# Andhra Pradesh bus details

In [39]:
# URL of the website
URL = "https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    driver = initialize_driver()  # Initialize the driver once

    try:
        for page in range(1,3):  # There are 3 pages
            try:
                load_page(driver, URL)
                
                if page > 1:
                    pagination_tab = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                    )
                    driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                    driver.execute_script("arguments[0].click();", pagination_tab)
                    time.sleep(5)  # Wait for the page to load
                
                all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
                # Iterate over each bus route link and scrape the details
                for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                    bus_details = scrape_bus_details(driver, link, name)
                    if bus_details:
                        all_bus_details.extend(bus_details)
            except Exception as e:
                print(f"Error occurred while accessing page {page}: {str(e)}")

    finally:
        driver.quit()  # Close the driver at the end

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('C:/Users/adith/Downloads/red_bus_data_2.csv', index=False)

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF789F7B125+29573]
	(No symbol) [0x00007FF789EEFF50]
	(No symbol) [0x00007FF789DAB6EA]
	(No symbol) [0x00007FF789DFF815]
	(No symbol) [0x00007FF789DFFA6C]
	(No symbol) [0x00007FF789E4B917]
	(No symbol) [0x00007FF789E2733F]
	(No symbol) [0x00007FF789E486BC]
	(No symbol) [0x00007FF789E270A3]
	(No symbol) [0x00007FF789DF12DF]
	(No symbol) [0x00007FF789DF2441]
	GetHandleVerifier [0x00007FF78A2AC76D+3377613]
	GetHandleVerifier [0x00007FF78A2F7B67+3685831]
	GetHandleVerifier [0x00007FF78A2ECF8B+3641835]
	GetHandleVerifier [0x00007FF78A03B2A6+816390]
	(No symbol) [0x00007FF789EFB25F]
	(No symbol) [0x00007FF789EF7084]
	(No symbol) [0x00007FF789EF7220]
	(No symbol) [0x00007FF789EE607F]
	BaseThreadInitThunk [0x00007FFCBADD257D+29]
	RtlUserThreadStart [0x00007FFCBC66AF28+40]



In [40]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,FRESHBUS,Electric A/C Seater (2+2),21:50,07h 45m,05:35,4.6,600,13 Seats available
1,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,IntrCity SmartBus,A/C Seater / Sleeper (2+1),22:45,06h 50m,05:35,4.4,INR 630,17 Seats available
2,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,IntrCity SmartBus,Scania AC Multi Axle Sleeper (2+1),23:00,07h 30m,06:30,4.5,INR 825,17 Seats available
3,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,AdIntrCity SmartBus,A/C Seater / Sleeper (2+1),23:15,07h 10m,06:25,4.0,INR 697,18 Seats available
4,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,Shyamoli Paribahan Pvt Ltd,Scania Multi-Axle AC Semi Sleeper (2+2),22:15,06h 55m,05:10,4.6,INR 649,18 Seats available
...,...,...,...,...,...,...,...,...,...,...
533,Bangalore to Rayachoti,https://www.redbus.in/bus-tickets/bangalore-to...,AR & BCVR Travels,A/C Sleeper (2+1),20:40,06h 50m,03:30,3.7,INR 1000,14 Seats available
534,Bangalore to Rayachoti,https://www.redbus.in/bus-tickets/bangalore-to...,SLNS Travels,A/C Sleeper (2+1),21:00,05h 30m,02:30,3.0,INR 1100,18 Seats available
535,Bangalore to Rayachoti,https://www.redbus.in/bus-tickets/bangalore-to...,Harikrishna Travels,Non A/C Seater / Sleeper (2+1),21:30,05h 05m,02:35,3.0,INR 4999,32 Seats available
536,Bangalore to Rayachoti,https://www.redbus.in/bus-tickets/bangalore-to...,Sri Mukundha Travels,NON A/C Semi Sleeper / Sleeper (2+1),22:30,04h 30m,03:00,2.8,INR 3999,38 Seats available


# Telungana

In [10]:
# URL of the website
URL = "https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    driver = initialize_driver()  # Initialize the driver once

    try:
        for page in range(1,3):  # There are 3 pages
            try:
                load_page(driver, URL)
                
                if page > 1:
                    pagination_tab = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                    )
                    driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                    driver.execute_script("arguments[0].click();", pagination_tab)
                    time.sleep(5)  # Wait for the page to load
                
                all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
                # Iterate over each bus route link and scrape the details
                for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                    bus_details = scrape_bus_details(driver, link, name)
                    if bus_details:
                        all_bus_details.extend(bus_details)
            except Exception as e:
                print(f"Error occurred while accessing page {page}: {str(e)}")

    finally:
        driver.quit()  # Close the driver at the end

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('C:/Users/adith/Downloads/red_bus_data_3.csv', index=False)

Error occurred while accessing https://www.redbus.in/bus-tickets/hyderabad-to-adilabad: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=129.0.6668.71)
Stacktrace:
	GetHandleVerifier [0x00007FF74B6DB125+29573]
	(No symbol) [0x00007FF74B64FF50]
	(No symbol) [0x00007FF74B50B6EA]
	(No symbol) [0x00007FF74B5036ED]
	(No symbol) [0x00007FF74B4F4459]
	(No symbol) [0x00007FF74B4F6202]
	(No symbol) [0x00007FF74B4F471F]
	(No symbol) [0x00007FF74B4F3FAB]
	(No symbol) [0x00007FF74B4F3EEA]
	(No symbol) [0x00007FF74B4F1D65]
	(No symbol) [0x00007FF74B4F23DC]
	(No symbol) [0x00007FF74B50E821]
	(No symbol) [0x00007FF74B5A94DE]
	(No symbol) [0x00007FF74B5872FA]
	(No symbol) [0x00007FF74B5A86BC]
	(No symbol) [0x00007FF74B5870A3]
	(No symbol) [0x00007FF74B5512DF]
	(No symbol) [0x00007FF74B552441]
	GetHandleVerifier [0x00007FF74BA0C76D+3377613]
	GetHandleVerifier [0x00007FF74BA57B67+3685831]
	GetHandleVerifier [0x00007FF74BA4CF8B+3641835]
	GetHandleVerifier [0x00007FF74B79B2A6+816

In [11]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,FRESHBUS,Electric A/C Seater (2+2),15:30,07h 40m,23:10,4.7,530,24 Seats available
1,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,FRESHBUS,Electric A/C Seater (2+2),23:10,06h 25m,05:35,4.6,600,15 Seats available
2,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,FRESHBUS,Electric A/C Seater (2+2),12:45,06h 40m,19:25,4.5,450,18 Seats available
3,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,IntrCity SmartBus,A/C Seater / Sleeper (2+1),23:50,05h 45m,05:35,4.4,INR 735,22 Seats available
4,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,IntrCity SmartBus,Scania AC Multi Axle Sleeper (2+1),23:59,06h 31m,06:30,4.5,INR 978,20 Seats available
...,...,...,...,...,...,...,...,...,...,...
626,Hyderabad to Bhadrachalam,https://www.redbus.in/bus-tickets/hyderabad-to...,Gupta Travels Galaxy,A/C Seater / Sleeper (2+2),21:00,07h 00m,04:00,1.8,INR 500,38 Seats available
627,Hyderabad to Bhadrachalam,https://www.redbus.in/bus-tickets/hyderabad-to...,Sai Krishna Bus Service,NON A/C Seater / Sleeper (2+2),21:15,06h 35m,03:50,1.4,656,43 Seats available
628,Hyderabad to Bhadrachalam,https://www.redbus.in/bus-tickets/hyderabad-to...,Maa Bankeshwari -Gupta Travels Galaxy,NON A/C Seater / Sleeper (2+2),19:15,07h 15m,02:30,1.9,INR 500,45 Seats available
629,Hyderabad to Bhadrachalam,https://www.redbus.in/bus-tickets/hyderabad-to...,A.R.M.T Co.,A/C Sleeper (2+1),17:30,07h 00m,00:30,1.8,INR 1200,22 Seats available


# Kadamba bus details

In [12]:
# URL of the website
URL = "https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    driver = initialize_driver()  # Initialize the driver once

    try:
        for page in range(1,3):  # There are 3 pages
            try:
                load_page(driver, URL)
                
                if page > 1:
                    pagination_tab = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                    )
                    driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                    driver.execute_script("arguments[0].click();", pagination_tab)
                    time.sleep(5)  # Wait for the page to load
                
                all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
                # Iterate over each bus route link and scrape the details
                for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                    bus_details = scrape_bus_details(driver, link, name)
                    if bus_details:
                        all_bus_details.extend(bus_details)
            except Exception as e:
                print(f"Error occurred while accessing page {page}: {str(e)}")

    finally:
        driver.quit()  # Close the driver at the end

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('C:/Users/adith/Downloads/red_bus_data_4.csv', index=False)

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/goa-to-pune: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF74B6DB125+29573]
	(No symbol) [0x00007FF74B64FF50]
	(No symbol) [0x00007FF74B50B6EA]
	(No symbol) [0x00007FF74B55F815]
	(No symbol) [0x00007FF74B55FA6C]
	(No symbol) [0x00007FF74B5AB917]
	(No symbol) [0x00007FF74B58733F]
	(No symbol) [0x00007FF74B5A86BC]
	(No symbol) [0x00007FF74B5870A3]
	(No symbol) [0x00007FF74B5512DF]
	(No symbol) [0x00007FF74B552441]
	GetHandleVerifier [0x00007FF74BA0C76D+3377613]
	GetHandleVerifier [0x00007FF74BA57B67+3685831]
	GetHandleVerifier [0x00007FF74BA4CF8B+3641835]
	GetHandleVerifier [0x00007FF74B79B2A6+816390]
	(No symbol) [0x00007FF74B65B25F]
	(No symbol) [0x00007FF74B657084]
	(No symbol) [0x00007FF74B657220]
	(No symbol) [0x00007FF74B64607F]
	BaseThreadInitThunk [0x00007FFCBADD257D+29]
	RtlUserThreadStart [0x00007FFCBC66AF28+40]



In [13]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,IntrCity SmartBus,Bharat Benz A/C Sleeper (2+1),22:30,10h 15m,08:45,4.2,INR 2074,9 Seats available
1,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Ashray Vip Yellows,A/C Sleeper (2+1),20:30,11h 30m,08:00,4.0,INR 1930,11 Seats available
2,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Laxmi-Princess Nova,Non A/C Seater / Sleeper (2+1),20:00,11h 15m,07:15,4.2,INR 999,3 Seats available
3,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,AdGogte Shreyas,A/C Sleeper (2+1),21:45,12h 15m,10:00,4.0,INR 1699,6 Seats available
4,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,IntrCity SmartBus,Bharat Benz A/C Seater /Sleeper (2+1),21:00,11h 05m,08:05,4.4,INR 1576,6 Seats available
...,...,...,...,...,...,...,...,...,...,...
307,Goa to Mopa Airport,https://www.redbus.in/bus-tickets/goa-to-mopa-...,Kadamba Transport Corporation Limited (KTCL) -...,A/C Seater (2+3),15:55,01h 05m,17:00,2.5,INR 200,48 Seats available
308,Goa to Mopa Airport,https://www.redbus.in/bus-tickets/goa-to-mopa-...,Kadamba Transport Corporation Limited (KTCL) -...,A/C Seater (2+3),17:14,00h 31m,17:45,3.7,INR 150,40 Seats available
309,Goa to Miraj,https://www.redbus.in/bus-tickets/goa-to-miraj,Kadamba Transport Corporation Limited (KTCL) -...,Non AC Seater 2+2,19:00,09h 30m,04:30,3.2,INR 505,20 Seats available
310,Goa to Miraj,https://www.redbus.in/bus-tickets/goa-to-miraj,URL Travels,A/C Sleeper (2+1),13:05,05h 25m,18:30,2.6,INR 1999,30 Seats available


# Rajasthan bus details

In [14]:
# URL of the website
URL = "https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    driver = initialize_driver()  # Initialize the driver once

    try:
        for page in range(1,3):  # There are 3 pages
            try:
                load_page(driver, URL)
                
                if page > 1:
                    pagination_tab = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                    )
                    driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                    driver.execute_script("arguments[0].click();", pagination_tab)
                    time.sleep(5)  # Wait for the page to load
                
                all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
                # Iterate over each bus route link and scrape the details
                for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                    bus_details = scrape_bus_details(driver, link, name)
                    if bus_details:
                        all_bus_details.extend(bus_details)
            except Exception as e:
                print(f"Error occurred while accessing page {page}: {str(e)}")

    finally:
        driver.quit()  # Close the driver at the end

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('C:/Users/adith/Downloads/red_bus_data_5.csv', index=False)

In [15]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,RSRTC - 192831,Express Non AC Seater 2+3,15:00,06h 00m,21:00,4.2,INR 285,42 Seats available
1,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,RSRTC - 193271,Express Non AC Seater 2+3,19:30,06h 30m,02:00,4.2,INR 294,34 Seats available
2,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,RSRTC - 189622,Express Non AC Seater 2+3,22:00,07h 00m,05:00,3.5,INR 294,42 Seats available
3,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,M R Travels,Volvo Multi-Axle I-Shift B11R Semi Sleeper (2+2),17:10,04h 45m,21:55,4.4,INR 600,20 Seats available
4,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,Jain travels regd,A/C Seater / Sleeper (2+1),21:30,07h 00m,04:30,4.2,INR 460,28 Seats available
...,...,...,...,...,...,...,...,...,...,...
669,Jaipur (Rajasthan) to Kota(Rajasthan),https://www.redbus.in/bus-tickets/jaipur-to-ko...,Geetanjali Travels,NON AC Seater / Sleeper 2+1,23:00,05h 30m,04:30,1.6,INR 800,15 Seats available
670,Jaipur (Rajasthan) to Kota(Rajasthan),https://www.redbus.in/bus-tickets/jaipur-to-ko...,Navrang Travels,A/C Sleeper (2+1),23:30,06h 00m,05:30,0,760,22 Seats available
671,Jaipur (Rajasthan) to Kota(Rajasthan),https://www.redbus.in/bus-tickets/jaipur-to-ko...,Ajay Raj Travel Agency,A/C Seater / Sleeper (2+1),22:30,05h 00m,03:30,0,600,36 Seats available
672,Jaipur (Rajasthan) to Kota(Rajasthan),https://www.redbus.in/bus-tickets/jaipur-to-ko...,A One Travel Agency,A/C Sleeper (2+1),21:15,04h 55m,02:10,0,INR 899,32 Seats available


# South bengal bus details

In [16]:
# URL of the website
URL = "https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    driver = initialize_driver()  # Initialize the driver once

    try:
        for page in range(1,3):  # There are 3 pages
            try:
                load_page(driver, URL)
                
                if page > 1:
                    pagination_tab = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                    )
                    driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                    driver.execute_script("arguments[0].click();", pagination_tab)
                    time.sleep(5)  # Wait for the page to load
                
                all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
                # Iterate over each bus route link and scrape the details
                for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                    bus_details = scrape_bus_details(driver, link, name)
                    if bus_details:
                        all_bus_details.extend(bus_details)
            except Exception as e:
                print(f"Error occurred while accessing page {page}: {str(e)}")

    finally:
        driver.quit()  # Close the driver at the end

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('C:/Users/adith/Downloads/red_bus_data_6.csv', index=False)

In [17]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-ASANSOL - KOLKATA - L/S - 12:35 (HOWRAH ...,Non AC Seater (2+3),13:40,03h 30m,17:10,3.0,INR 150,29 Seats available
1,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-DURGAPUR - HABRA - VIA - DUNLOP - 14:05 ...,Non AC Seater (2+3),14:05,03h 30m,17:35,4.2,INR 155,41 Seats available
2,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-ASANSOL - KARUNAMOYEE - VOLVO - 13:25 (B...,Volvo AC Seater (2+2),14:30,04h 15m,18:45,4.3,INR 360,49 Seats available
3,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-BANKURA - KARUNAMOYEE - 13:00 (BNK DEPOT...,Non AC Seater (2+3),14:55,04h 00m,18:55,3.7,INR 155,47 Seats available
4,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-ASANSOL - KARUNAMOYEE - 13:55 (ASANSOL D...,Non AC Seater (2+3),15:05,04h 00m,19:05,2.8,INR 155,41 Seats available
...,...,...,...,...,...,...,...,...,...,...
421,Digha to Durgapur (West Bengal),https://www.redbus.in/bus-tickets/digha-to-dur...,SBSTC-DIGHA - DURGAPUR - VIA - BISHNUPUR - 2/2...,Non AC Seater (2+2),20:30,08h 10m,04:40,3.7,INR 233,28 Seats available
422,Digha to Durgapur (West Bengal),https://www.redbus.in/bus-tickets/digha-to-dur...,SBSTC-DIGHA - ASANSOL - VIA - MEDINIPUR-DURGAP...,Non AC Seater (2+3),21:30,08h 30m,06:00,3.2,INR 238,41 Seats available
423,Midnapore to Barasat (West Bengal),https://www.redbus.in/bus-tickets/midnapore-to...,WBTC (CTC) Habra-Midnapur - 194|14:45,Non AC Seater (2+3),14:45,03h 10m,17:55,3.8,INR 120,42 Seats available
424,Midnapore to Barasat (West Bengal),https://www.redbus.in/bus-tickets/midnapore-to...,WBTC (CTC) Habra-Midnapur - 194|15:40,Non AC Seater (2+3),15:40,03h 15m,18:55,3.8,INR 120,33 Seats available


# Himachal Pradesh bus details

In [18]:
# URL of the website
URL = "https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    driver = initialize_driver()  # Initialize the driver once

    try:
        for page in range(1,3):  # There are 3 pages
            try:
                load_page(driver, URL)
                
                if page > 1:
                    pagination_tab = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                    )
                    driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                    driver.execute_script("arguments[0].click();", pagination_tab)
                    time.sleep(5)  # Wait for the page to load
                
                all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
                # Iterate over each bus route link and scrape the details
                for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                    bus_details = scrape_bus_details(driver, link, name)
                    if bus_details:
                        all_bus_details.extend(bus_details)
            except Exception as e:
                print(f"Error occurred while accessing page {page}: {str(e)}")

    finally:
        driver.quit()  # Close the driver at the end

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('C:/Users/adith/Downloads/red_bus_data_7.csv', index=False)

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/delhi-to-baddi-himachal-pradesh: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF74B6DB125+29573]
	(No symbol) [0x00007FF74B64FF50]
	(No symbol) [0x00007FF74B50B6EA]
	(No symbol) [0x00007FF74B55F815]
	(No symbol) [0x00007FF74B55FA6C]
	(No symbol) [0x00007FF74B5AB917]
	(No symbol) [0x00007FF74B58733F]
	(No symbol) [0x00007FF74B5A86BC]
	(No symbol) [0x00007FF74B5870A3]
	(No symbol) [0x00007FF74B5512DF]
	(No symbol) [0x00007FF74B552441]
	GetHandleVerifier [0x00007FF74BA0C76D+3377613]
	GetHandleVerifier [0x00007FF74BA57B67+3685831]
	GetHandleVerifier [0x00007FF74BA4CF8B+3641835]
	GetHandleVerifier [0x00007FF74B79B2A6+816390]
	(No symbol) [0x00007FF74B65B25F]
	(No symbol) [0x00007FF74B657084]
	(No symbol) [0x00007FF74B657220]
	(No symbol) [0x00007FF74B64607F]
	BaseThreadInitThunk [0x00007FFCBADD257D+29]
	RtlUserThreadStart [0x00007FFCBC66AF28+40]



In [19]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 93,Himsuta AC Seater Volvo/Scania 2+2,19:35,09h 10m,04:45,3.6,INR 912,3 Seats available
1,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 591,A/C Executive (2+3),21:00,10h 20m,07:20,3.7,INR 632,20 Seats available
2,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 570,Ordinary 3+2 Non AC Seater,21:25,09h 20m,06:45,2.6,INR 512,35 Seats available
3,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 9,Himsuta AC Seater Volvo/Scania 2+2,22:00,09h 30m,07:30,4.3,INR 912,4 Seats available
4,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 36,Himsuta AC Seater Volvo/Scania 2+2,22:30,08h 00m,06:30,4.1,INR 887,1 Seat available
...,...,...,...,...,...,...,...,...,...,...
323,Chandigarh to Kangra,https://www.redbus.in/bus-tickets/chandigarh-t...,HRTC - 91,Himsuta AC Seater Volvo/Scania 2+2,22:10,05h 55m,04:05,4.1,INR 715,7 Seats available
324,Chandigarh to Kangra,https://www.redbus.in/bus-tickets/chandigarh-t...,HRTC - 1213,Ordinary 3+2 Non AC Seater,22:47,06h 43m,05:30,3.9,INR 410,10 Seats available
325,Chandigarh to Kangra,https://www.redbus.in/bus-tickets/chandigarh-t...,Laxmi holidays,VE A/C Semi Sleeper (2+2),23:50,05h 00m,04:50,4.8,759,16 Seats available
326,Chandigarh to Kangra,https://www.redbus.in/bus-tickets/chandigarh-t...,Laxmi holidays,Volvo 9600 A/C Semi Sleeper (2+2),23:55,05h 15m,05:10,4.5,1234,6 Seats available


# Assam bus details

In [41]:
# URL of the website
URL = "https://www.redbus.in/online-booking/astc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    driver = initialize_driver()  # Initialize the driver once

    try:
        for page in range(1,3):  # There are 3 pages
            try:
                load_page(driver, URL)
                
                if page > 1:
                    pagination_tab = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                    )
                    driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                    driver.execute_script("arguments[0].click();", pagination_tab)
                    time.sleep(5)  # Wait for the page to load
                
                all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
                # Iterate over each bus route link and scrape the details
                for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                    bus_details = scrape_bus_details(driver, link, name)
                    if bus_details:
                        all_bus_details.extend(bus_details)
            except Exception as e:
                print(f"Error occurred while accessing page {page}: {str(e)}")

    finally:
        driver.quit()  # Close the driver at the end

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('C:/Users/adith/Downloads/red_bus_data_8.csv', index=False)

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/nagaon-to-guwahati: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF789F7B125+29573]
	(No symbol) [0x00007FF789EEFF50]
	(No symbol) [0x00007FF789DAB6EA]
	(No symbol) [0x00007FF789DFF815]
	(No symbol) [0x00007FF789DFFA6C]
	(No symbol) [0x00007FF789E4B917]
	(No symbol) [0x00007FF789E2733F]
	(No symbol) [0x00007FF789E486BC]
	(No symbol) [0x00007FF789E270A3]
	(No symbol) [0x00007FF789DF12DF]
	(No symbol) [0x00007FF789DF2441]
	GetHandleVerifier [0x00007FF78A2AC76D+3377613]
	GetHandleVerifier [0x00007FF78A2F7B67+3685831]
	GetHandleVerifier [0x00007FF78A2ECF8B+3641835]
	GetHandleVerifier [0x00007FF78A03B2A6+816390]
	(No symbol) [0x00007FF789EFB25F]
	(No symbol) [0x00007FF789EF7084]
	(No symbol) [0x00007FF789EF7220]
	(No symbol) [0x00007FF789EE607F]
	BaseThreadInitThunk [0x00007FFCBADD257D+29]
	RtlUserThreadStart [0x00007FFCBC66AF28+40]

Error occurred while scraping bus details for https://www.redbus.in/bus-tic

In [42]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,BLUE HILL TRAVELS INDIA LIMITED,NON A/C Seater Semi Sleeper (2+1),22:15,05h 30m,03:45,3.5,450,5 Seats available
1,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Himalayan Travels,NON AC Seater / Sleeper 2+1,23:40,04h 20m,04:00,2.2,INR 500,16 Seats available
2,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,Yatra Travels,NON A/C Seater (2+1),18:00,05h 30m,23:30,3.2,400,12 Seats available
3,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,Yatra Travels,NON AC Seater/ Sleeper (2+1),20:00,04h 00m,00:00,3.5,400,7 Seats available
4,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,KIRAN TRAVELS,A/C Seater (2+1),20:40,03h 50m,00:30,3.1,500,7 Seats available
5,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,Pranjit Travels (Under ASTC),A/C Seater / Sleeper (2+1),20:45,05h 45m,02:30,3.2,INR 500,14 Seats available
6,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,Pranjit Travels (Under ASTC),NON A/C Seater (2+1),21:15,04h 30m,01:45,3.2,INR 450,17 Seats available
7,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,Pranjit Travels (Under ASTC),Bharat Benz A/C Seater (2+1),21:30,05h 00m,02:30,4.1,510,7 Seats available
8,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,Pranjit Travels (Under ASTC),Non AC Seater (2+1),21:45,04h 15m,02:00,3.0,INR 450,27 Seats available
9,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,DESTINY TRAVELS,NON A/C Seater (2+1),18:45,05h 20m,00:05,1.9,425,31 Seats available


# Uttar Pradesh bus details

In [28]:
# URL of the website
URL = "https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    driver = initialize_driver()  # Initialize the driver once

    try:
        for page in range(1,3):  # There are 3 pages
            try:
                load_page(driver, URL)
                
                if page > 1:
                    pagination_tab = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                    )
                    driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                    driver.execute_script("arguments[0].click();", pagination_tab)
                    time.sleep(5)  # Wait for the page to load
                
                all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
                # Iterate over each bus route link and scrape the details
                for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                    bus_details = scrape_bus_details(driver, link, name)
                    if bus_details:
                        all_bus_details.extend(bus_details)
            except Exception as e:
                print(f"Error occurred while accessing page {page}: {str(e)}")

    finally:
        driver.quit()  # Close the driver at the end

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('C:/Users/adith/Downloads/red_bus_data_9.csv', index=False)

In [29]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,UPSRTC - BLY0148,Rajdhani Non AC Seater 2+3,17:00,05h 00m,22:00,3.3,INR 448,52 Seats available
1,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,UPSRTC - SRG0229,Ordinary Non AC Seater 2+3,17:00,05h 35m,22:35,3.3,INR 418,52 Seats available
2,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,UPSRTC - BRT0036,Ordinary Non AC Seater 2+3,17:00,07h 10m,00:10,3.3,INR 449,52 Seats available
3,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,UPSRTC - BLY0107,Janrath AC Seater 2+2,17:02,04h 43m,21:45,1.9,INR 598,35 Seats available
4,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,UPSRTC - HDD0090,Ordinary Non AC Seater 2+3,17:04,09h 46m,02:50,3.3,INR 444,52 Seats available
...,...,...,...,...,...,...,...,...,...,...
567,Lucknow to Varanasi,https://www.redbus.in/bus-tickets/lucknow-to-v...,UPSRTC - HDD0001,Janrath AC Seater 2+2,23:30,08h 00m,07:30,1.8,INR 698,36 Seats available
568,Lucknow to Varanasi,https://www.redbus.in/bus-tickets/lucknow-to-v...,UPSRTC - CNT0097,Ordinary Non AC Seater 2+3,23:31,07h 59m,07:30,3.3,INR 480,52 Seats available
569,Lucknow to Varanasi,https://www.redbus.in/bus-tickets/lucknow-to-v...,PTC-SKYBUS,A/C Seater/Sleeper (2+1),22:30,07h 00m,05:30,4.7,INR 399,22 Seats available
570,Lucknow to Varanasi,https://www.redbus.in/bus-tickets/lucknow-to-v...,Mahalaxmi Travels,Bharat Benz A/C Seater /Sleeper (2+1),23:15,06h 30m,05:45,3.5,INR 381,18 Seats available


# West Bengal bus details

In [45]:
# URL of the website
URL = "https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    driver = initialize_driver()  # Initialize the driver once

    try:
        for page in range(1,3):  # There are 3 pages
            try:
                load_page(driver, URL)
                
                if page > 1:
                    pagination_tab = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                    )
                    driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                    driver.execute_script("arguments[0].click();", pagination_tab)
                    time.sleep(5)  # Wait for the page to load
                
                all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
                # Iterate over each bus route link and scrape the details
                for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                    bus_details = scrape_bus_details(driver, link, name)
                    if bus_details:
                        all_bus_details.extend(bus_details)
            except Exception as e:
                print(f"Error occurred while accessing page {page}: {str(e)}")

    finally:
        driver.quit()  # Close the driver at the end

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('C:/Users/adith/Downloads/red_bus_data_10.csv', index=False)

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/barasat-west-bengal-to-digha: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF789F7B125+29573]
	(No symbol) [0x00007FF789EEFF50]
	(No symbol) [0x00007FF789DAB6EA]
	(No symbol) [0x00007FF789DFF815]
	(No symbol) [0x00007FF789DFFA6C]
	(No symbol) [0x00007FF789E4B917]
	(No symbol) [0x00007FF789E2733F]
	(No symbol) [0x00007FF789E486BC]
	(No symbol) [0x00007FF789E270A3]
	(No symbol) [0x00007FF789DF12DF]
	(No symbol) [0x00007FF789DF2441]
	GetHandleVerifier [0x00007FF78A2AC76D+3377613]
	GetHandleVerifier [0x00007FF78A2F7B67+3685831]
	GetHandleVerifier [0x00007FF78A2ECF8B+3641835]
	GetHandleVerifier [0x00007FF78A03B2A6+816390]
	(No symbol) [0x00007FF789EFB25F]
	(No symbol) [0x00007FF789EF7084]
	(No symbol) [0x00007FF789EF7220]
	(No symbol) [0x00007FF789EE607F]
	BaseThreadInitThunk [0x00007FFCBADD257D+29]
	RtlUserThreadStart [0x00007FFCBC66AF28+40]

Error occurred while scraping bus details for https://www.redbus.

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/midnapore-to-barasat-west-bengal: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF789F7B125+29573]
	(No symbol) [0x00007FF789EEFF50]
	(No symbol) [0x00007FF789DAB6EA]
	(No symbol) [0x00007FF789DFF815]
	(No symbol) [0x00007FF789DFFA6C]
	(No symbol) [0x00007FF789E4B917]
	(No symbol) [0x00007FF789E2733F]
	(No symbol) [0x00007FF789E486BC]
	(No symbol) [0x00007FF789E270A3]
	(No symbol) [0x00007FF789DF12DF]
	(No symbol) [0x00007FF789DF2441]
	GetHandleVerifier [0x00007FF78A2AC76D+3377613]
	GetHandleVerifier [0x00007FF78A2F7B67+3685831]
	GetHandleVerifier [0x00007FF78A2ECF8B+3641835]
	GetHandleVerifier [0x00007FF78A03B2A6+816390]
	(No symbol) [0x00007FF789EFB25F]
	(No symbol) [0x00007FF789EF7084]
	(No symbol) [0x00007FF789EF7220]
	(No symbol) [0x00007FF789EE607F]
	BaseThreadInitThunk [0x00007FFCBADD257D+29]
	RtlUserThreadStart [0x00007FFCBC66AF28+40]

Error occurred while scraping bus details for https://www.red

In [46]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,The Joy Ride,A/C Seater / Sleeper (2+2),21:15,06h 20m,03:35,3.7,488,51 Seats available
1,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,Aradhana Travels,A/C Seater / Sleeper (2+2),18:35,05h 20m,23:55,1.1,450,19 Seats available
2,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,Satya Paribahan,A/C Seater (2+3),23:00,06h 10m,05:10,2.8,INR 280,30 Seats available
3,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,Aradhana Bus Service,A/C Seater / Sleeper (2+2),23:00,05h 30m,04:30,2.1,INR 550,75 Seats available
4,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,Snemita Paribahan (Shinjini),AC Seater (2+3),23:35,05h 25m,05:00,2.4,INR 330,33 Seats available
5,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,Arunita Paribahan,A/C Seater (2+3),23:35,05h 25m,05:00,2.8,INR 370,21 Seats available
6,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,SHYAMOLI Pvt. Ltd.(Karunamoyee),Volvo A/C Seater (2+2),17:50,03h 45m,21:35,4.2,INR 486,18 Seats available
7,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,Express Line,Volvo 9600 Multi Axle Semi-Sleeper (2+2),18:30,04h 20m,22:50,4.5,INR 486,16 Seats available
8,Digha to Calcutta,https://www.redbus.in/bus-tickets/digha-to-kol...,Snemita Paribahan(Shinjini),A/C Seater (2+3),22:45,05h 25m,04:10,3.6,INR 320,22 Seats available
9,Digha to Calcutta,https://www.redbus.in/bus-tickets/digha-to-kol...,City Express (Shinjini),AC Seater (2+3),22:45,05h 25m,04:10,3.5,INR 290,26 Seats available


In [5]:
df1 = pd.read_csv("red_bus_data_1.csv")
df2 = pd.read_csv("red_bus_data_2.csv")
df3 = pd.read_csv("red_bus_data_3.csv")
df4 = pd.read_csv("red_bus_data_4.csv")
df5 = pd.read_csv("red_bus_data_5.csv")
df6 = pd.read_csv("red_bus_data_6.csv")
df7 = pd.read_csv("red_bus_data_7.csv")
df8 = pd.read_csv("red_bus_data_8.csv")
df9 = pd.read_csv("red_bus_data_9.csv")
df10 = pd.read_csv("red_bus_data_10.csv")

In [6]:
df_buses = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10], ignore_index = True)
df_buses

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 235,Swift Deluxe Non AC Air Bus (2+2),15:00,09h 01m,00:01,3.9,INR 801,5 Seats available
1,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 370,AC MULTI AXLE,15:25,06h 55m,22:20,2.7,INR 1099,6 Seats available
2,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 97,Swift Deluxe Non AC Air Bus (2+2),16:00,08h 45m,00:45,2.4,INR 740,4 Seats available
3,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 171,Swift Deluxe Non AC Air Bus (2+2),16:15,09h 30m,01:45,4.1,INR 832,2 Seats available
4,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 553,Super Express Non AC Seater Air Bus (2+2),16:45,08h 40m,01:25,4.1,INR 832,3 Seats available
...,...,...,...,...,...,...,...,...,...,...
4225,Kolkata to Durgapur (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-d...,Greenline,Volvo A/C B11R Multi Axle Semi Sleeper (2+2),18:05,04h 00m,22:05,4.4,INR 486,19 Seats available
4226,Kolkata to Durgapur (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-d...,Shyamoli Paribahan Pvt Ltd,Volvo A/C Seater (2+2),18:40,03h 45m,22:25,4.5,INR 486,12 Seats available
4227,Kolkata to Durgapur (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-d...,Royal Cruiser,Scania Multi-Axle AC Semi Sleeper (2+2),19:30,03h 45m,23:15,3.4,463,20 Seats available
4228,Kolkata to Durgapur (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-d...,Pariwartan Bus Service,NON A/C Seater / Sleeper (2+2),18:30,04h 30m,23:00,2.7,INR 500,54 Seats available


In [10]:
path = "C:/Users/adith/Downloads/red_bus_details_data.csv"
df_buses.to_csv(path,index = False)

In [9]:
# Data preprocessing 

df_buses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4230 entries, 0 to 4229
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Route_Name         4230 non-null   object 
 1   Route_Link         4230 non-null   object 
 2   Bus_Name           4230 non-null   object 
 3   Bus_Type           4230 non-null   object 
 4   Departing_Time     4230 non-null   object 
 5   Duration           4230 non-null   object 
 6   Reaching_Time      4230 non-null   object 
 7   Star_Rating        4230 non-null   float64
 8   Price              4230 non-null   float64
 9   Seat_Availability  4230 non-null   object 
dtypes: float64(2), object(8)
memory usage: 330.6+ KB


In [8]:
#converting prices to numeric

df_buses["Price"] = df_buses["Price"].str.replace("INR","")
df_buses["Price"] = df_buses["Price"].astype(float)
df_buses["Price"].fillna(0)

0        801.0
1       1099.0
2        740.0
3        832.0
4        832.0
         ...  
4225     486.0
4226     486.0
4227     463.0
4228     500.0
4229     740.0
Name: Price, Length: 4230, dtype: float64